In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install torch torchvision
!pip install opencv-python
!pip install matplotlib

In [ ]:
import os
import csv
import torch
from PIL import Image
from transformers import AutoImageProcessor, Dinov2Model
from tqdm import tqdm

In [5]:
# Set up the model and image processor
model_name = "facebook/dinov2-large"
image_processor = AutoImageProcessor.from_pretrained(model_name)
model = Dinov2Model.from_pretrained(model_name)

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to process a single image
def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = image_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the CLS token embedding (first token of the last hidden state)
    features = outputs.last_hidden_state[:, 0, :].cpu().numpy().flatten()
    return features

# Set the input folder and output CSV file
input_folder = "/content/drive/MyDrive/Cassava/Data 2/train_images"
output_csv = "output_features.csv"

# Get all image files from the input folder
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

# Process images and write features to CSV
with open(output_csv, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write header
    header = ['image_name'] + [f'feature_{i}' for i in range(model.config.hidden_size)]
    csv_writer.writerow(header)

    # Process each image and write features
    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(input_folder, image_file)
        features = process_image(image_path)

        # Write the image name and its features to the CSV
        row = [image_file] + list(features)
        csv_writer.writerow(row)

print(f"Features extracted and saved to {output_csv}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Processing images: 100%|██████████| 13/13 [00:12<00:00,  1.05it/s]

Features extracted and saved to output_features.csv
